In [1]:
import pandas as pd

In [2]:
cols = ["frame", "x", "y", "obj","size","seq","tbd1","tbd2","tbd3","filename","start","path_time","delta_time","tbd4"]
useful_cols = ["frame", "x", "y", "obj","size","seq","filename","start","path_time","delta_time"]

In [ ]:
%time points = pd.read_csv("data/fixed.csv",names=cols,usecols=useful_cols,parse_dates=['start'])

In [ ]:
points.info(memory_usage='deep')

In [ ]:
# points.info()
# len(points)
# points.describe()
points.head(50)

In [3]:
p = points
# p.groupby(p.start).size()

NameError: name 'points' is not defined

In [ ]:
for dtype in ['int64','object']:
    selected_dtype = p.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

In [4]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else: # we assume if not a df it's a series
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b / 1024 ** 2 # convert bytes to megabytes
    return "{:03.2f} MB".format(usage_mb)

In [ ]:
p_int = points.select_dtypes(include=['int64'])
converted_int = p_int.apply(pd.to_numeric,downcast='unsigned')
print(mem_usage(p_int))
print(mem_usage(converted_int))

In [ ]:
compare_ints = pd.concat([p_int.dtypes,converted_int.dtypes],axis=1)
compare_ints.columns = ['before','after']
compare_ints.apply(pd.Series.value_counts)

In [ ]:
optimized_p = p.copy()

optimized_p[converted_int.columns] = converted_int

print(mem_usage(p))
print(mem_usage(optimized_p))

In [ ]:
dtypes = optimized_p.dtypes

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

# rather than print all 161 items, we'll
# sample 10 key/value pairs from the dict
# and print it nicely using prettyprint

preview = first2pairs = {key:value for key,value in list(column_types.items())[:10]}
import pprint
pp = pp = pprint.PrettyPrinter(indent=4)
pp.pprint(preview)

In [ ]:
p_obj = p.select_dtypes(include=['object']).copy()
p_obj.describe()

In [ ]:
fn = p_obj.filename
fn_cat = fn.astype('category')
print(fn_cat.head())


In [ ]:
fn_cat.head().cat.codes

In [ ]:
print(mem_usage(fn))
print(mem_usage(fn_cat))


In [ ]:
converted_obj = pd.DataFrame()

for col in p_obj.columns:
    num_unique_values = len(p_obj[col].unique())
    num_total_values = len(p_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = p_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = p_obj[col]

In [ ]:
optimized_p[converted_obj.columns] = converted_obj

mem_usage(optimized_p)

In [ ]:
print(mem_usage(p_obj))
print(mem_usage(converted_obj))

compare_obj = pd.concat([p_obj.dtypes,converted_obj.dtypes],axis=1)
compare_obj.columns = ['before','after']
compare_obj.apply(pd.Series.value_counts)

In [ ]:
converted_obj.info(memory_usage='deep')

In [ ]:
dtypes = optimized_p.dtypes

dtypes_col = dtypes.index
dtypes_type = [i.name for i in dtypes.values]

column_types = dict(zip(dtypes_col, dtypes_type))

# rather than print all 161 items, we'll
# sample 10 key/value pairs from the dict
# and print it nicely using prettyprint

preview = first2pairs = {key:value for key,value in list(column_types.items())[:10]}
import pprint
pp = pp = pprint.PrettyPrinter(indent=4)
pp.pprint(preview)

In [5]:
cols_types = dict({   
    'delta_time': 'object',
    'filename': 'category',
    'frame': 'uint16',
    'obj': 'uint16',
    'path_time': 'object',
    'seq': 'uint16',
    'size': 'uint32',
    'x': 'uint16',
    'y': 'uint16'})

In [6]:
%%time 
read_and_optimized = pd.read_csv('data/fixed.csv',names=cols,usecols=useful_cols,dtype=cols_types,parse_dates=['start'],infer_datetime_format=True)

print(mem_usage(read_and_optimized))
read_and_optimized.head()

283.10 MB
Wall time: 25 s


In [8]:
p = read_and_optimized
len(p)
p.head(5)

,frame,x,y,obj,size,seq,filename,start,path_time,delta_time
0,0,14,329,0,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00
1,0,4,291,1,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00
2,0,629,7,2,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00
3,0,253,4,3,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00
4,0,185,4,4,362,0,14-16\20170817012709_100.h264.txt,2017-08-17 01:27:09,01:27:09,0:00:00


In [30]:
u_p = p.groupby(['filename','obj','x','y']).size()

In [32]:
len(u_p)

1324447

In [21]:
mem_usage(u_p)

'20.27 MB'

In [26]:
u_p

filename                            obj  x    y    seq
 14-16\20170817012709_100.h264.txt  0    14   329  0        1
                                    1    4    291  0        1
                                    2    629  7    0        1
                                    3    253  4    0        1
                                    4    185  4    0        1
                                    5    5    102  0        1
                                         6    102  1        1
                                                   2        1
                                         7    102  3        1
                                         9    102  4        1
                                         11   102  5       61
                                    6    76   100  0        1
                                         80   100  1        1
                                         94   100  2        5
                                         109  101  3        1
               

In [23]:
p[(p.obj == 0) & (p.filename == "14-16\20170817012709_100.h264.txt")]

,frame,x,y,obj,size,seq,filename,start,path_time,delta_time
